In [1]:
import copy
import importlib
import os
import warnings
from itertools import product

import torch
from dotenv import load_dotenv
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.tuner import Tuner
from tqdm.auto import tqdm

load_dotenv("../env")

warnings.filterwarnings("ignore", ".*does not have many workers.*")

In [2]:
import luminar.document.data
import luminar.document.model
import luminar.features
import luminar.mongo

importlib.reload(luminar.document.data)
importlib.reload(luminar.document.model)
importlib.reload(luminar.features)
importlib.reload(luminar.mongo)

<module 'luminar.mongo' from '/nvme/projects/PrismAI/PrismAI/src/luminar/mongo.py'>

In [3]:
from luminar.document.data import (
    DocumentClassificationDataModule,
    FeatureDataset,
    PaddingDataloader,
    n_way_split,
)
from luminar.document.model import ConvolutionalLayerSpec, DocumentClassficationModel
from luminar.features import FeatureExtractor, OneDimFeatures, Slicer, TwoDimFeatures
from luminar.mongo import PrismaiDataset

In [4]:
domains = {
    "Blog Authorship": {"domain": "blog_authorship_corpus"},
    "Student Essays": {"domain": "student_essays"},
    "CNN News": {"domain": "cnn_news"},
    "Euro Court Cases": {"domain": "euro_court_cases"},
    "House of Commons": {"domain": "house_of_commons"},
    "ArXiv Papers": {"domain": "arxiv_papers"},
    "Gutenberg": {"domain": "gutenberg", "lang": "en-EN"},
    "Bundestag [DE]": {"domain": "bundestag"},
    "Spiegel [DE]": {"domain": "spiegel_articles"},
    # "Gutenberg [DE]": {"domain": "gutenberg", "lang": "de-DE"},
}

In [5]:
dmap = {
    "Blog Authorship": "Blog Authorship",
    "Student Essays": "Student Essays",
    "CNN News": "CNN News",
    "Euro Court Cases": "Euro Court Cases",
    "House of Commons": "House of Commons",
    "ArXiv Papers": "ArXiv Papers",
    "Gutenberg [EN]": "Gutenberg",
    "Bundestag": "Bundestag [DE]",
    "Spiegel": "Spiegel [DE]",
    # "Gutenberg [DE]": 
}

In [6]:
config = {
    "seed": 1337,
    "eval_split": 0.1,
    "test_split": 0.1,
    "feature_model": "gpt2",
    "synth_agent": "gpt-4o-mini",
    # "synth_agent": "gemma2:9b"
}

In [7]:
datasets = {
    domain: PrismaiDataset(
        mongo_db_connection=os.environ.get("MONGO_DB_CONNECTION"),
        database="prismai",
        collection="features_prismai",
        feature_model=config["feature_model"],
        synth_agent=config["synth_agent"],
        **kwargs,
    )
    for domain, kwargs in domains.items()
}

In [8]:
train_splits = {}
eval_splits = {}
test_splits = {}
for domain, dataset in datasets.items():
    seed_everything(config["seed"])
    train_dataset, eval_dataset, test_dataset = n_way_split(
        dataset,
        config["eval_split"],
        config["test_split"],
        infer_first=True,
    )
    train_splits[domain] = train_dataset
    eval_splits[domain] = eval_dataset
    test_splits[domain] = test_dataset

Seed set to 1337
[PrismaiDataset] Loading Documents from MongoDB: 1500it [00:03, 464.53it/s]


[PrismaiDataset] Writing Cache File /nvme/.cache/luminar/ef0bf4ac00353b4c3cf35f1d81719f7c755a0aa74285110bc6d705aeab0f4275.pkl


Seed set to 1337
[PrismaiDataset] Loading Documents from MongoDB: 1500it [00:12, 124.47it/s]


[PrismaiDataset] Writing Cache File /nvme/.cache/luminar/5486290edc393c9f30480a59dd534ea751ae8a31b48bfd03b5f098c4f8bdc1cc.pkl


Seed set to 1337
[PrismaiDataset] Loading Documents from MongoDB: 1500it [00:18, 82.97it/s]


...

[PrismaiDataset] Writing Cache File /nvme/.cache/luminar/f38cb33ea5c3f40856a4db07115946d7a99c95fca0b54ce148415fb5063be266.pkl


Seed set to 1337
[PrismaiDataset] Loading Documents from MongoDB: 1492it [00:26, 57.04it/s]


[PrismaiDataset] Writing Cache File /nvme/.cache/luminar/281b270f73261fde5b5517abe476f7282592e6e4c0c4161857d3fd83c33aaf9b.pkl


Seed set to 1337
[PrismaiDataset] Loading Documents from MongoDB: 1500it [00:19, 78.75it/s]


[PrismaiDataset] Writing Cache File /nvme/.cache/luminar/50f2f9ce260cd153c5bca0c86708073a7221aad4c39b121601fdbb01dc3e6d31.pkl


In [9]:
# feature_dim = OneDimFeatures(256)
# featurizer = FeatureExtractor.Likelihood()
# featurizer = FeatureExtractor.LogLikelihoodLogRankRatio()
# config["second_dim_as_channels"] = False
feature_dim = TwoDimFeatures(256, 13)
# featurizer = FeatureExtractor.LikelihoodTopkLikelihoodRatio(13)
featurizer = FeatureExtractor.IntermediateLogits(13)
config["second_dim_as_channels"] = True

# slicer = Slicer.Random(feature_dim[0])
slicer = Slicer.RandomMultiple(feature_dim[0] // 4, multiple=4, stride=16)
# slicer = Slicer.RandomMultiple(feature_dim[0] // 4, 4)

config["feature_dim"] = feature_dim
config["featurizer"] = repr(featurizer)
config["slicer"] = repr(slicer)

config["num_samples"] = None


def featurize(dataset) -> FeatureDataset:
    return FeatureDataset(
        dataset, slicer, featurizer, num_samples=config["num_samples"]
    )

In [10]:
train_datasets, eval_datasets, test_datasets = {}, {}, {}
for domain, dataset in tqdm(datasets.items()):
    train_datasets[domain] = featurize(train_splits[domain])
    eval_datasets[domain] = featurize(eval_splits[domain])
    test_datasets[domain] = featurize(test_splits[domain])

  0%|          | 0/9 [00:00<?, ?it/s]

## In-Domain Training & Evaluation

In [11]:
config |= {
    "projection_dim": None,
    "learning_rate": 0.0001,
    "warmup_steps": 66,
    "max_epochs": 50,
    "gradient_clip_val": 0.5,
    "batch_size": 32,
}

# SeqXGPT Layer Configuration
config["conv_layer_shapes"] = [
    ConvolutionalLayerSpec(64, 5),
    *[ConvolutionalLayerSpec(128, 3)] * 3,
    ConvolutionalLayerSpec(64, 3),
]

In [12]:
from torch.utils.data import ConcatDataset


# capturing config from "closure"
def get_dataloader(*dataset, **kwargs) -> PaddingDataloader:
    if len(dataset) == 1:
        dataset = dataset[0]
    else:
        dataset = ConcatDataset(dataset)
    return PaddingDataloader(
        dataset,
        feature_dim=config["feature_dim"],
        batch_size=config["batch_size"],
        **kwargs,
    )

In [13]:
results_in_domain = {}
for domain in domains:
    seed_everything(config["seed"])

    train_dataloader = get_dataloader(train_datasets[domain], shuffle=True)
    eval_dataloader = get_dataloader(eval_datasets[domain])

    model = DocumentClassficationModel(**config)
    trainer = Trainer(
        max_epochs=config["max_epochs"],
        logger=pl_loggers.TensorBoardLogger(
            save_dir=f"logs/in_domain/{type(featurizer).__name__}",
            name=domain,
        ),
        gradient_clip_val=config["gradient_clip_val"],
        callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=3)],
        deterministic=True,
    )

    trainer.fit(
        model,
        train_dataloaders=train_dataloader,
        val_dataloaders=eval_dataloader,
    )

    trainer.progress_bar_callback.disable()
    metrics = []
    for other, dataset in test_datasets.items():
        trainer.validate(model, get_dataloader(eval_datasets[other]), verbose=False)
        metrics.append(
            {
                "other": other,
            }
            | trainer.test(
                model,
                get_dataloader(dataset),
                verbose=False,
            )[0]
        )
    trainer.progress_bar_callback.enable()

    results_in_domain[domain] = {
        "domain": domain,
        "config": copy.deepcopy(config),
        "metrics": metrics,
    }
    print(domain, metrics)


Seed set to 1337
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:45: Attribute 'feature_dim' removed from hparams because it cannot be pickled. You can suppress this warning by setting `self.save_hyperparameters(ignore=['feature_dim'])`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name        | Type              | Params | Mode  | In sizes    | Out sizes  
--------------------------------------------------------------------------------------
0 | conv_layers | Sequential        | 152 K  | train | ?           | ?          
1 | projection  | Identity          | 0      | train | [32, 16384] | [32, 16384]
2 | classifier  | Linear            | 16.4 K | train | [32, 16384] | [32, 1]    
3 | criterion   | BCEWithLogitsLoss | 0      | train | ?   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Spiegel [DE] [{'other': 'Blog Authorship', 'test_loss': 1.899004578590393, 'test_acc@0.5': 0.5300353169441223, 'test_acc@best': 0.554770290851593, 'test_roc_auc': 0.3709774315357208}, {'other': 'Student Essays', 'test_loss': 1.9004229307174683, 'test_acc@0.5': 0.6133333444595337, 'test_acc@best': 0.6233333349227905, 'test_roc_auc': 0.6586666703224182}, {'other': 'CNN News', 'test_loss': 1.2783634662628174, 'test_acc@0.5': 0.6633333563804626, 'test_acc@best': 0.6700000166893005, 'test_roc_auc': 0.740066647529602}, {'other': 'Euro Court Cases', 'test_loss': 1.8682434558868408, 'test_acc@0.5': 0.6033333539962769, 'test_acc@best': 0.6800000071525574, 'test_roc_auc': 0.7562888860702515}, {'other': 'House of Commons', 'test_loss': 1.1434385776519775, 'test_acc@0.5': 0.6677852272987366, 'test_acc@best': 0.6744966506958008, 'test_roc_auc': 0.7288288474082947}, {'other': 'ArXiv Papers', 'test_loss': 0.9927137494087219, 'test_acc@0.5': 0.7337883710861206, 'test_acc@best': 0.7406143546104431, 'te

In [14]:
import pandas as pd

results = []
for domain in domains:
    results.append(
        [results_in_domain[domain]["metrics"][i]["test_roc_auc"] for i in range(len(domains))]
    )

df = pd.DataFrame(results, columns=domains, index=domains)
df

,Blog Authorship,Student Essays,CNN News,Euro Court Cases,House of Commons,ArXiv Papers,Gutenberg,Bundestag [DE],Spiegel [DE]
Blog Authorship,0.990526,0.670311,0.770356,0.727311,0.680946,0.818656,0.830402,0.585027,0.671365
Student Essays,0.545363,0.987378,0.871422,0.864267,0.789640,0.765632,0.874610,0.712686,0.776286
CNN News,0.242957,0.795467,0.981600,0.946978,0.846487,0.856025,0.904823,0.728708,0.656823
Euro Court Cases,0.266541,0.822844,0.931867,0.994267,0.912973,0.919765,0.871915,0.569288,0.500940
House of Commons,0.428672,0.918933,0.903600,0.921378,0.974189,0.935887,0.882033,0.713489,0.653870
ArXiv Papers,0.570501,0.891022,0.897956,0.909911,0.887252,0.979499,0.911111,0.603696,0.497852
Gutenberg,0.368120,0.893822,0.861822,0.844978,0.925450,0.944041,0.977116,0.718074,0.716868
Bundestag [DE],0.195764,0.658844,0.810867,0.593467,0.688333,0.751864,0.787612,0.954627,0.927696
Spiegel [DE],0.370977,0.658667,0.740067,0.756289,0.728829,0.787205,0.687092,0.892476,0.953468


## Cross-Domain Training & Evaluation

In [15]:
seed_everything(config["seed"])
train_dataloader = get_dataloader(*train_datasets.values(), shuffle=True)
eval_dataloader = get_dataloader(*eval_datasets.values())
test_dataloader = get_dataloader(*test_datasets.values())

model = DocumentClassficationModel(**config)
trainer = Trainer(
    max_epochs=config["max_epochs"],
    logger=pl_loggers.TensorBoardLogger(
        save_dir=f"logs/all_domains/{type(featurizer).__name__}",
    ),
    gradient_clip_val=config["gradient_clip_val"],
    callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=3)],
    deterministic=True,
)
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=eval_dataloader,
)

Seed set to 1337
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/mastoeck/Projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:45: Attribute 'feature_dim' removed from hparams because it cannot be pickled. You can suppress this warning by setting `self.save_hyperparameters(ignore=['feature_dim'])`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name        | Type              | Params | Mode  | In sizes    | Out sizes  
--------------------------------------------------------------------------------------
0 | conv_layers | Sequential        | 152 K  | train | ?           | ?          
1 | projection  | Identity          | 0      | train | [32, 16384] | [32, 16384]
2 | classifier  | Linear            | 16.4 K | train | [32, 16384] | [32, 1]    
3 | criterion   | BCEWithLogitsLoss | 0      | train | ?   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [16]:
trainer.progress_bar_callback.disable()
metrics_cross_domain = []
for other, dataset in test_datasets.items():
    trainer.validate(model, get_dataloader(eval_datasets[domain]), verbose=False)
    metrics_cross_domain.append(
        {"other": other}
        | trainer.test(
            model,
            get_dataloader(dataset),
            verbose=False,
        )[0]
    )
trainer.validate(model, eval_dataloader, verbose=False)
metrics_cross_domain += [
    {
        "other": "ALL",
        **trainer.test(
            model,
            test_dataloader,
            verbose=False,
        )[0],
    }
]
trainer.progress_bar_callback.enable()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
import pandas as pd


def df_to_latex_heatmap(_df: pd.DataFrame):
    print(
        "\\plotHeatmap{"
        + ",".join(_df.index)
        + "}{%\n    "
        + ",%\n    ".join(
            [
                "{"
                + ",".join(f"{val:.4f}/{round(val, 2):.2f}" for val in row[1:])
                + "}"
                for row in _df.reset_index().values
            ]
        )
        + "%\n}{"
        + ",".join(_df.columns)
        + "}"
    )


In [18]:
# _metric = "test_roc_auc"
_metric = "test_acc"

results = []
for domain in domains:
    results.append(
        [
            results_in_domain[domain]["metrics"][i][_metric]
            for i in range(len(domains))
        ]
    )
results.append([m[_metric] for m in metrics_cross_domain[:len(domains)]])

df = pd.DataFrame(results, columns=domains, index=list(domains) + ["ALL"])
df["AVG"] = df.mean(axis=1)
df["AVG"][-1] = metrics_cross_domain[-1][_metric]
df

KeyError: 'test_acc'

In [ ]:
df_to_latex_heatmap(df)

In [ ]:
config

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

ax = sns.heatmap(
    df,
    annot=True,
    fmt=".2f",
    vmax=1.0,
    vmin=0.0,
    cmap=sns.cubehelix_palette(rot=-0.2, as_cmap=True),
    yticklabels=list(domains) + ["ALL"],
    xticklabels=list(domains) + ["AVG"],
    square=True,
    # reduce annotation font size
    annot_kws={"fontsize": 8},
    cbar=False,
)

# rotate x-axis labels by 45 degrees
# anchored at the right edge of the axes
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment("right")

plt.tight_layout()
# plt.savefig(
#     "../figures/evaluation-trained_in_domain-test_0.1-gpt2_256-rand_4-il_13_as_channels.pdf",
#     dpi=300,
# )
plt.show()

### LLR on Whole Datasets

In [ ]:
from sklearn.metrics import auc, roc_curve

from luminar.baselines import llr_from_transition_scores
from simple_dataset import Dataset as SimpleDataset
from transition_scores.data import TransitionScores

results_llr = []
for domain, split in datasets.items():
    dataset_test = (
        SimpleDataset(split)
        .flat_map(lambda doc: doc["features"])
        .map(
            lambda x: {
                "llr": llr_from_transition_scores(
                    TransitionScores(**x["transition_scores"])
                ),
                "labels": int(x["type"] != "source"),
            },
            in_place=False,
        )
    )
    fpr, tpr, _ = roc_curve(dataset_test["labels"], dataset_test["llr"])
    auroc = auc(fpr, tpr)

    preds = np.array(dataset_test["llr"])
    labels = np.array(dataset_test["labels"])

    mean_0 = float(np.mean(preds[labels == 0]))
    mean_1 = float(np.mean(preds[labels == 1]))

    thresholds = np.linspace(round(mean_0, 1) - 0.2, round(mean_1, 1) + 0.3, 5001)
    preds_thresholded: np.ndarray = preds > thresholds.reshape(-1, 1)
    acc_thresholded = np.mean((preds_thresholded == labels), axis=1)
    idx = np.argmax(acc_thresholded)
    best_threshold = thresholds[idx]
    best_acc = acc_thresholded[idx]

    results_llr.append({
        "domain": domain,
        "auroc": auroc,
        "best_acc": best_acc,
        "best_threshold": best_threshold,
    })

pd.DataFrame(results_llr)

### LLR on Test Splits

In [ ]:
from sklearn.metrics import auc, roc_curve

from luminar.baselines import llr_from_transition_scores
from simple_dataset import Dataset as SimpleDataset
from transition_scores.data import TransitionScores

results_llr = []
for domain, split in test_splits.items():
    dataset_test = (
        SimpleDataset(split)
        .flat_map(lambda doc: doc["features"])
        .map(
            lambda x: {
                "llr": llr_from_transition_scores(
                    TransitionScores(**x["transition_scores"])
                ),
                "labels": int(x["type"] != "source"),
            },
            in_place=False,
        )
    )
    fpr, tpr, _ = roc_curve(dataset_test["labels"], dataset_test["llr"])
    auroc = auc(fpr, tpr)

    preds = np.array(dataset_test["llr"])
    labels = np.array(dataset_test["labels"])

    mean_0 = float(np.mean(preds[labels == 0]))
    mean_1 = float(np.mean(preds[labels == 1]))

    thresholds = np.linspace(round(mean_0, 1) - 0.2, round(mean_1, 1) + 0.3, 1001)
    preds_thresholded: np.ndarray = preds > thresholds.reshape(-1, 1)
    acc_thresholded = np.mean((preds_thresholded == labels), axis=1)
    idx = np.argmax(acc_thresholded)
    best_threshold = thresholds[idx]
    best_acc = acc_thresholded[idx]

    results_llr.append({
        "domain": domain,
        "auroc": auroc,
        "best_acc": best_acc,
        "best_threshold": best_threshold,
    })

pd.DataFrame(results_llr)

In [ ]:
raise RuntimeError("STOP")

In [ ]:
sample = train_datasets["CNN News"][0]["features"].numpy().squeeze()
train_datasets["CNN News"][0]["labels"]

In [ ]:
fig = plt.imshow(sample.T, cmap=sns.cubehelix_palette(rot=-0.2, as_cmap=True))
fig.axes.set_axis_off()
plt.show()

In [ ]:
sample = train_datasets["CNN News"][1]["features"].numpy().squeeze()
train_datasets["CNN News"][1]["labels"]

In [ ]:
fig = plt.imshow(sample.T, cmap=sns.cubehelix_palette(rot=-0.2, as_cmap=True))
fig.axes.set_axis_off()
plt.show()

In [ ]:
raise RuntimeError("STOP")

In [ ]:
from sklearn.metrics import auc, roc_curve

from luminar.baselines import llr_from_transition_scores

dataset_test = dm._dataset_test.map(
    lambda x: {
        "llr": llr_from_transition_scores(x["features"]),
        "labels": x["labels"],
    },
    in_place=False,
)
fpr, tpr, _ = roc_curve(dataset_test["labels"], dataset_test["llr"])
auc(fpr, tpr)